Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Открыв его, увидим следующую структуру:

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:
http://pandas.pydata.org/pandas-docs/stable/cookbook.html

In [1]:
import pandas as pd

data = pd.read_csv("checkins.dat", header=0, skiprows=2, names=['id', 'user_id', 'venue_id','latitude', 'longitude', 'created_at'],  delimiter='|', skipinitialspace=True)

/home/sigor/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
data.head()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
1,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
2,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
3,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58
4,984268,2146843.0,5222.0,NaN,NaN,2012-04-21 17:42:38


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021966 entries, 0 to 1021965
Data columns (total 6 columns):
id            1021966 non-null object
user_id       1021965 non-null float64
venue_id      1021965 non-null float64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    1021965 non-null object
dtypes: float64(4), object(2)
memory usage: 46.8+ MB


In [4]:
d = data[:5]

In [17]:
d.loc[:1] # to row with index 1

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
1,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18


In [20]:
d[:2] # tow first rows

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
1,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18


In [21]:
d.dropna()

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
2,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43


Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [7]:
data = data.dropna()

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396634 entries, 0 to 1021963
Data columns (total 6 columns):
id            396634 non-null object
user_id       396634 non-null float64
venue_id      396634 non-null float64
latitude      396634 non-null float64
longitude     396634 non-null float64
created_at    396634 non-null object
dtypes: float64(4), object(2)
memory usage: 21.2+ MB


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.



In [16]:
data2 = data[:100000]
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 233797
Data columns (total 6 columns):
id            100000 non-null object
user_id       100000 non-null float64
venue_id      100000 non-null float64
latitude      100000 non-null float64
longitude     100000 non-null float64
created_at    100000 non-null object
dtypes: float64(4), object(2)
memory usage: 5.3+ MB


In [11]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth


In [26]:
d[['latitude','longitude']]

,latitude,longitude
0,38.895112,-77.036366
1,NaN,NaN
2,33.800745,-84.410520
3,NaN,NaN
4,NaN,NaN


In [27]:
bandwidth=0.1
ms = MeanShift(bandwidth)
ms.fit(data2[['latitude','longitude']])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [35]:
print len(ms.cluster_centers_), len(ms.labels_), ms.cluster_centers_[0], ms.labels_[0]

 3230 100000 [ 40.7177164  -73.99183542] 5


In [30]:
ms.cluster_centers_

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ..., 
       [  46.7323875 , -117.0001651 ],
       [  29.6899563 ,  -95.8996757 ],
       [  31.3787916 ,  -95.3213317 ]])

In [31]:
ms.labels_

array([ 5,  7, 30, ..., 25, 19,  4])

In [83]:
print sum(ms.labels_ == 0),sum(ms.labels_ == 1),sum(ms.labels_ == 2)

12506 4692 3994


In [84]:
l = pd.DataFrame(ms.labels_, columns=['cluster'])

In [85]:
l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 1 columns):
cluster    100000 non-null int64
dtypes: int64(1)
memory usage: 781.3 KB


In [86]:
l.sort_values(by=('cluster'), ascending=False)

,cluster
71788,3229
35163,3228
22841,3227
33230,3226
87847,3225
33643,3224
80744,3223
6384,3222
59629,3221
41408,3220


In [57]:
d.sort_values(by=('id'))

,id,user_id,venue_id,latitude,longitude,created_at
0,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
2,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
3,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58
4,984268,2146843.0,5222.0,NaN,NaN,2012-04-21 17:42:38
1,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18


In [74]:
dd = d.groupby('venue_id').size().reset_index(name='counts')

In [75]:
dd

,venue_id,counts
0,5222.0,5


In [87]:
res = l.groupby(['cluster']).size().reset_index(name='counts')

In [88]:
res.counts.sum()

100000L

In [89]:
res.head()

,cluster,counts
0,0,12506
1,1,4692
2,2,3994
3,3,3363
4,4,3526


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [99]:
res[res.counts > 15]

,cluster,counts
0,0,12506
1,1,4692
2,2,3994
3,3,3363
4,4,3526
5,5,2409
6,6,2297
7,7,1601
8,8,1526
9,9,1378


In [114]:
optimal_cluster_idx = res[res.counts > 15].index
optimal_cluster_idx[-2]

1268

In [107]:
optimal_clusters = ms.cluster_centers_[optimal_cluster_idx]

In [110]:
optimal_clusters.shape

(593, 2)

In [111]:
optimal_clusters

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ..., 
       [  41.61853175,  -88.44556818],
       [  38.65877915,  -76.8856871 ],
       [  39.2494686 ,  -77.1821271 ]])

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт mapcustomizer.com имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

In [116]:
for point in optimal_clusters:
    print point[0],",",point[1]

40.7177163973 , -73.9918354199
33.4494380502 , -112.00213969
33.4463802704 , -111.901887562
41.8782437797 , -87.6298433623
37.6886815741 , -122.409330374
38.8861652156 , -77.0487833307
33.3573445623 , -111.822654108
33.7666362322 , -84.3932891848
42.3632186398 , -71.0736876086
47.6062447174 , -122.332043826
36.117229143 , -115.171073423
34.0603975546 , -118.248709027
44.9779478203 , -93.2673008853
30.267183617 , -97.7431192813
40.76687624 , -73.8333534905
39.7358301526 , -104.986580428
39.951680373 , -75.1627359239
34.0354869531 , -118.438997719
32.9808933822 , -117.078117978
32.8030205353 , -96.7698974349
37.3478711439 , -121.947287224
28.5435015466 , -81.3764286229
32.7113444339 , -117.153638748
32.2217131518 , -110.926535153
34.1274022191 , -118.351883725
29.7626977547 , -95.3823137047
43.0405328168 , -87.914332566
33.8173064339 , -117.891249171
37.390292432 , -122.087286436
25.78581242 , -80.2179380368
45.5234832146 , -122.676280421
33.5697346035 , -112.314015838
33.6743026598 , -1

43.4515758148 , -80.4999384815
29.6195397923 , -95.6319761385
39.6945237885 , -75.7521574923
18.4339505077 , -66.1107003269
-34.6119959577 , -58.3718653577
41.9981228231 , -91.6697868692
-7.79746918462 , 110.366031742
42.1292241 , -80.085059
40.1210773538 , -75.4908632731
40.6940440923 , -89.5908347538
41.2033216 , -77.1945247
47.6587802 , -117.4260466
33.2078221115 , -97.1251833654
40.2862357385 , -111.688920623
37.2136113231 , -93.2966431038
32.9398771115 , -96.5904837115
40.6294525 , -75.49198334
42.087314624 , -88.284168884
30.282790376 , -81.395321336
34.6100243 , -112.315721
41.712166196 , -71.433390084
41.605020956 , -87.903899084
40.290735404 , -75.132053236
42.062429216 , -71.198006748
43.074149144 , -70.772261672
44.0247062 , -88.5426136
38.7809192917 , -90.534779225
39.755543 , -105.2210997
41.5898431167 , -83.6716027833
40.6473882708 , -74.602102525
30.0080668292 , -95.2366504625
36.59689935 , -121.892799779
37.9753390208 , -121.737315162
28.6641521875 , -81.1904694042
41.3

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса всех офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

In [133]:
offices = [
    [33.751277, -118.188740, "Los Angeles"],
    [25.867736, -80.324116,  "Miami"],
    [51.503016, -0.075479, "London"],
    [52.378894, 4.885084, "Amsterdam"],
    [39.366487, 117.036146, "Beijing"],
    [-33.868457, 151.205134, "Sydney"]
]

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

In [179]:
l1, l2, _ = zip(*offices)
points = zip(l1, l2)

In [181]:
from scipy.spatial import distance
distance.cdist(points, points, 'euclidean')

array([[   0.        ,   38.67660752,  119.43980351,  124.47551675,
         235.29189866,  277.75076557],
       [  38.67660752,    0.        ,   84.24376132,   89.23816035,
         197.82135703,  239.11128447],
       [ 119.43980351,   84.24376132,    0.        ,    5.03729566,
         117.73881283,  173.70697243],
       [ 124.47551675,   89.23816035,    5.03729566,    0.        ,
         112.9034253 ,  169.84746859],
       [ 235.29189866,  197.82135703,  117.73881283,  112.9034253 ,
           0.        ,   80.81384017],
       [ 277.75076557,  239.11128447,  173.70697243,  169.84746859,
          80.81384017,    0.        ]])

In [263]:
dist = distance.cdist(points, optimal_clusters) # get matirx with distances between every points

In [183]:
dist.shape

(6, 593)

In [195]:
dist

array([[  44.74257092,    6.19395917,    6.29424146, ...,   30.76605219,
          41.5935783 ,   41.37357142],
       [  16.14371999,   32.5726786 ,   32.47447468, ...,   17.72133043,
          13.24513413,   13.74564883],
       [  74.69906582,  113.37331719,  113.2748328 , ...,   88.92117683,
          77.87671339,   78.07422497],
       [  79.73425538,  118.41008135,  118.31160891, ...,   93.94890117,
          82.91381403,   83.1108233 ],
       [ 191.0327603 ,  229.11470442,  229.01456477, ...,  205.49405482,
         193.92312447,  194.21830835],
       [ 237.22725876,  271.67953165,  271.58164907, ...,  251.2583223 ,
         239.34415107,  239.80611351]])

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [197]:
a = np.arange(6).reshape(2,3)
a

array([[0, 1, 2],
       [3, 4, 5]])

In [198]:
a.reshape(1,6)

array([[0, 1, 2, 3, 4, 5]])

In [213]:
b = a.reshape(1,a.shape[0]*a.shape[1])[0]
b

array([0, 1, 2, 3, 4, 5])

In [215]:
b.reshape(a.shape[0],a.shape[1])

array([[0, 1, 2],
       [3, 4, 5]])

In [222]:
import operator  # import itemgetter, attrgetter
sorted(enumerate(b), key=operator.itemgetter(1) )[:3]

[(0, 0), (1, 1), (2, 2)]

In [225]:
dest_array = dist.reshape(1, dist.shape[0]*dist.shape[1])[0] # make one array for sorting

In [237]:
dest_min_20 = sorted(enumerate(dest_array), key=operator.itemgetter(1))[:20] # sort with index saving
dest_min_20


[(3375, 0.007834758163107856),
 (2151, 0.0093533161859922255),
 (995, 0.022674066158385495),
 (1244, 0.050058294822787869),
 (51, 0.070847732427199731),
 (622, 0.13410903336184654),
 (759, 0.16740596425035326),
 (685, 0.18887596060185083),
 (87, 0.19577945647763628),
 (42, 0.21181053682436798),
 (285, 0.22223329073179071),
 (908, 0.27130075950667348),
 (119, 0.29497888680045692),
 (648, 0.30227011869246051),
 (27, 0.30473050307840693),
 (11, 0.31488379033627317),
 (32, 0.33881047025113181),
 (751, 0.34084565332205718),
 (17, 0.37868750125029754),
 (47, 0.38670622484272771)]

In [242]:
print "module",11//4,"remainder",11%4, 3375//dist.shape[1]

module 2 remainder 3 5


In [261]:
def index_restore(num, shape_m): # restore indexes into matrix
    row = num // shape_m 
    col = num % shape_m 
    return [row, col]

In [259]:
dist.shape[1]

593

In [262]:
for d in dest_min_20:
    row, col = index_restore(d[0], dist.shape[1])
    print d, dist.shape[1], row, col #print restored matrix indexes

(3375, 0.007834758163107856) 593 5 410
(2151, 0.0093533161859922255) 593 3 372
(995, 0.022674066158385495) 593 1 402
(1244, 0.050058294822787869) 593 2 58
(51, 0.070847732427199731) 593 0 51
(622, 0.13410903336184654) 593 1 29
(759, 0.16740596425035326) 593 1 166
(685, 0.18887596060185083) 593 1 92
(87, 0.19577945647763628) 593 0 87
(42, 0.21181053682436798) 593 0 42
(285, 0.22223329073179071) 593 0 285
(908, 0.27130075950667348) 593 1 315
(119, 0.29497888680045692) 593 0 119
(648, 0.30227011869246051) 593 1 55
(27, 0.30473050307840693) 593 0 27
(11, 0.31488379033627317) 593 0 11
(32, 0.33881047025113181) 593 0 32
(751, 0.34084565332205718) 593 1 158
(17, 0.37868750125029754) 593 0 17
(47, 0.38670622484272771) 593 0 47


In [248]:
for d in dest_min_20:
    #row, col = index_restore(d[0], dist.shape[1])
    print d, dist.shape[1], d[0]//dist.shape[1], d[0]% dist.shape[1] #, row, col

(3375, 0.007834758163107856) 593 5 410
(2151, 0.0093533161859922255) 593 3 372
(995, 0.022674066158385495) 593 1 402
(1244, 0.050058294822787869) 593 2 58
(51, 0.070847732427199731) 593 0 51
(622, 0.13410903336184654) 593 1 29
(759, 0.16740596425035326) 593 1 166
(685, 0.18887596060185083) 593 1 92
(87, 0.19577945647763628) 593 0 87
(42, 0.21181053682436798) 593 0 42
(285, 0.22223329073179071) 593 0 285
(908, 0.27130075950667348) 593 1 315
(119, 0.29497888680045692) 593 0 119
(648, 0.30227011869246051) 593 1 55
(27, 0.30473050307840693) 593 0 27
(11, 0.31488379033627317) 593 0 11
(32, 0.33881047025113181) 593 0 32
(751, 0.34084565332205718) 593 1 158
(17, 0.37868750125029754) 593 0 17
(47, 0.38670622484272771) 593 0 47


In [249]:
dist[5][410] # get value off

0.007834758163107856

In [250]:
optimal_clusters[410]

array([ -33.86063043,  151.20477593])